<a href="https://colab.research.google.com/github/Hitchiker-V/Breaking-NNs-and-Backprop/blob/Part-1/Andrej_Karpathy_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# def f(x):
#   return 3*x**2 - 4*x + 5

In [4]:
# f(3.0)

In [5]:
# xs = np.arange(-5,5,0.25)
# ys = f(xs)
# plt.plot(xs, ys)

In [6]:
# Visualizing operation through GraphWiz

from graphviz import Digraph

def trace(root):
  # building unique nodes and edges
  nodes, edges = set(), set()
  def build(v): # v denotes Value objects
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def node_id(n):
  return str(id(n))

def draw_dot(root):
  dot = Digraph(format = 'svg', graph_attr={'rankdir': 'LR'})

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    dot.node(name = uid, label = "{%s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape = 'record')
    if n._op:
      # if node a result of some op, make an op node
      dot.node(name = uid + n._op, label = n._op)
      # connect op node to the prev node that was operated on
      dot.edge(uid + n._op, uid)
    
  for n1, n2 in edges:
      # connect n1 to op node of n2
      dot.edge(node_id(n1), node_id(n2) + n2._op)

  return dot

In [7]:
#building micrograd's Value object and internal operators

# class Value:
#   def __init__(self, data):
#     self.data = data
  
#   def __repr__(self):
#     return f"Value(data={self.data})"
  
#   def __add__(self, other):
#     return Value(self.data + other.data)
  
#   def __mul__(self, other):
#     return Value(self.data*other.data)


# a = Value(3.0)
# b = Value(-7.5)
# c = Value(10.0)

# d = a*b + c

# To implement above function, we need any add operator to include previous operation results, hence making changes to above. Also keeping track of which variable resulted from which final operator
# class Value:
#   def __init__(self, data, _children = (), _op='', label =''):
#     self.data = data
#     self._prev = set(_children) # using set to optmize for efficiency
#     self._op = _op 
#     self.label = label
#   def __repr__(self):
#     return f"Value(data={self.data})"
  
#   def __add__(self, other):
#     return Value(self.data + other.data, (self, other), '+')
  
#   def __mul__(self, other):
#     return Value(self.data*other.data, (self, other), '*')
  
# a = Value(3.0, label = 'a')
# b = Value(-7.5, label = 'b')
# c = Value(10.0, label = 'c')
# e = a*b; e.label = 'e'
# d = e + c; d.label = 'd'
# f = Value(-2.0, label = 'f')
# L = d*f; L.label = 'L'

In [8]:
# Implementing gradient
# class Value:
#   def __init__(self, data, _children = (), _op='', label =''):
#     self.data = data
#     self.grad = 0 # At initial there is no impact on the variables when data changed
#     self._prev = set(_children) # using set to optmize for efficiency
#     ## definining backprop step
#     self._backward = lambda : None
#     self._op = _op 
#     self.label = label
#   def __repr__(self):
#     return f"Value(data={self.data})"
  
#   def __add__(self, other):
#     out = Value(self.data + other.data, (self, other), '+')

#     def _backward():
#       self.grad = 1.0*out.grad
#       other.grad = 1.0*out.grad
#     out._backward = _backward
    
#     return out
  
#   def __mul__(self, other):
#     out = Value(self.data*other.data, (self, other), '*')

#     def _backward():
#       self.grad = other.data * out.grad
#       other.grad = self.data * out.grad
#     out._backward = _backward

#     return out

#   # for activation function
#   def tanh(self):
#     n = self.data
#     t = (math.exp(2*n)-1)/(math.exp(2*n)+1)
#     out = Value(t, (self, ), 'tanh')

#     def _backward():
#       self.grad = (1-t**2) * out.grad
#     out._backward = _backward

#     return out

# a = Value(3.0, label = 'a')
# b = Value(-7.5, label = 'b')
# c = Value(10.0, label = 'c')
# e = a*b; e.label = 'e'
# d = e + c; d.label = 'd'
# f = Value(-2.0, label = 'f')
# L = d*f; L.label = 'L'

In [9]:
# draw_dot(L)
# grad visualizes the derivative of L wrt to L, f, d, e, c, b, a respectively

In [10]:
# ## Implementing on a neuron

# x1 = Value(2.0, label = 'x1')
# x2 = Value(0.0, label = 'x2')
# w1 = Value(-3.0, label = 'w1')
# w2 = Value(1.0, label = 'w2')
# b = Value(6.8813735870195432, label = 'b')

# x1w1 = x1*w1; x1w1.label = 'x1w1'
# x2w2 = x2*w2; x2w2.label = 'x2w2'
# x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'

# n = x1w1x2w2 + b; n.label = 'n'
# o = n.tanh(); o.label = 'o'
# draw_dot(o)


In [11]:
# o.grad = 1.0

In [12]:
# o._backward()

In [13]:
# draw_dot(o)

In [14]:
# n._backward()
# draw_dot(o)

In [15]:
# x1w1x2w2._backward()
# draw_dot(o)

In [16]:
# x1w1._backward()
# x2w2._backward()
# draw_dot(o)

In [17]:
# To implement backprop, the propogation should travel from top to botton : Using Topological Sort for it

# o.grad = 1.0

# topo = [] # Empty array containing the nodes in right order
# visited = set() 
# def build_topo(v):
#   if v not in visited:
#    visited.add(v)
#    for child in v._prev:
#      build_topo(child)
#    topo.append(v)

# build_topo(o)
# topo

# ## Then implement _backward() method on this array sorted in topological order
# for node in reversed(topo):
#   node._backward()

# draw_dot(o)

In [18]:
# Wrapping inside the value class itself

#Buggy Implementation
# class Value:
#   def __init__(self, data, _children = (), _op='', label =''):
#     self.data = data
#     self.grad = 0 # At initial there is no impact on the variables when data changed
#     self._prev = set(_children) # using set to optmize for efficiency
#     ## definining backprop step
#     self._backward = lambda : None
#     self._op = _op 
#     self.label = label
#   def __repr__(self):
#     return f"Value(data={self.data})"
  
#   def __add__(self, other):
#     out = Value(self.data + other.data, (self, other), '+')

#     def _backward():
#       self.grad = 1.0*out.grad
#       other.grad = 1.0*out.grad
#     out._backward = _backward
    
#     return out
  
#   def __mul__(self, other):
#     out = Value(self.data*other.data, (self, other), '*')

#     def _backward():
#       self.grad = other.data * out.grad
#       other.grad = self.data * out.grad
#     out._backward = _backward

#     return out

#   # for activation function
#   def tanh(self):
#     n = self.data
#     t = (math.exp(2*n)-1)/(math.exp(2*n)+1)
#     out = Value(t, (self, ), 'tanh')

#     def _backward():
#       self.grad = (1-t**2) * out.grad
#     out._backward = _backward

#     return out
  
#   def backward(self): #topologically sorted back prop
#     topo = [] # Empty array containing the nodes in right order
#     visited = set() 
#     def build_topo(v):
#       if v not in visited:
#         visited.add(v)
#       for child in v._prev:
#         build_topo(child)
#       topo.append(v)

#     build_topo(self)

#     self.grad = 1.0
#     ## Then implement _backward() method on this array sorted in topological order
#     for node in reversed(topo):
#       node._backward()

In [19]:
## Implementing on a neuron

x1 = Value(2.0, label = 'x1')
x2 = Value(0.0, label = 'x2')
w1 = Value(-3.0, label = 'w1')
w2 = Value(1.0, label = 'w2')
b = Value(6.8813735870195432, label = 'b')

x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'

n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
draw_dot(o)

o.backward()
draw_dot(o)

NameError: ignored

In [ ]:
# There is a bug in the above implementation:
# a = Value(-2.0) ; a.label = 'a'
# b = Value(3.0) ; b.label = 'b'
# d = a + b ; d.label = 'd'
# e = a + b ; e.label = 'e'
# f = d + e ; f.label = 'f'

# c = a+b ; c.label = 'c'
# d = a*b ; d.label = 'd'
# e = c * d ; e.label = 'e'
# draw_dot(e) 
# f.backward()
# draw_dot(f)

b = a + a ; b.label = 'b'
b.backward()
draw_dot(b)
# Here a.grad should have been 2 but as in the backward(), we have self.grad = 1.0 + other.data, other.data = 1.0 + self.other
# If self.data and other.data are same variables as in this case, the grad doesn't get updated but rather replaced.

# Fix : Running summation of previous grads

In [ ]:
# Fixed implementation
# class Value:
#   def __init__(self, data, _children = (), _op='', label =''):
#     self.data = data
#     self.grad = 0 # At initial there is no impact on the variables when data changed
#     self._prev = set(_children) # using set to optmize for efficiency
#     ## definining backprop step
#     self._backward = lambda : None
#     self._op = _op 
#     self.label = label
#   def __repr__(self):
#     return f"Value(data={self.data})"
  
#   def __add__(self, other):
#     out = Value(self.data + other.data, (self, other), '+')

#     def _backward():
#       self.grad += 1.0*out.grad
#       other.grad += 1.0*out.grad
#     out._backward = _backward
    
#     return out
  
#   def __mul__(self, other):
#     out = Value(self.data*other.data, (self, other), '*')

#     def _backward():
#       self.grad += other.data * out.grad
#       other.grad += self.data * out.grad
#     out._backward = _backward

#     return out

#   # for activation function
#   def tanh(self):
#     n = self.data
#     t = (math.exp(2*n)-1)/(math.exp(2*n)+1)
#     out = Value(t, (self, ), 'tanh')

#     def _backward():
#       self.grad = (1-t**2) * out.grad
#     out._backward = _backward

#     return out
  
#   def backward(self): #topologically sorted back prop
#     topo = [] # Empty array containing the nodes in right order
#     visited = set() 
#     def build_topo(v):
#       if v not in visited:
#         visited.add(v)
#       for child in v._prev:
#         build_topo(child)
#       topo.append(v)

#     build_topo(self)

#     self.grad = 1.0
#     ## Then implement _backward() method on this array sorted in topological order
#     for node in reversed(topo):
#       node._backward()

In [ ]:
# a = Value(-2.0, label = 'a')
# b = a + a ; b.label = 'b'
# b.backward()
# draw_dot(b)

In [ ]:
# x1 = Value(2.0, label = 'x1')
# x2 = Value(0.0, label = 'x2')
# w1 = Value(-3.0, label = 'w1')
# w2 = Value(1.0, label = 'w2')
# b = Value(6.8813735870195432, label = 'b')

# x1w1 = x1*w1; x1w1.label = 'x1w1'
# x2w2 = x2*w2; x2w2.label = 'x2w2'
# x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'

# n = x1w1x2w2 + b; n.label = 'n'
# o = n.tanh(); o.label = 'o'
# draw_dot(o)

# o.backward()
# draw_dot(o)

In [44]:
# disecting tanh
# tanh = exp(2*x) - 1 / exp(2*x) + 1

# requires us to define exponential, constant multiplication, constant addition, subtraction, and division operations in Value class
class Value:
  def __init__(self, data, _children = (), _op='', label =''):
    self.data = data
    self.grad = 0 # At initial there is no impact on the variables when data changed
    self._prev = set(_children) # using set to optmize for efficiency
    ## definining backprop step
    self._backward = lambda : None
    self._op = _op 
    self.label = label
  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0*out.grad
      other.grad += 1.0*out.grad
    out._backward = _backward
    
    return out

  def __sub__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + (-1*other.data), (self, other), '-')

    def _backward():
        self.grad += -1.0*out.grad
        other.grad += -1.0*out.grad
    out._backward = _backward

    return out


  # caking care of a + 2 and 2 + a case
  def __radd__(self, other):
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data*other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

# caking care of a * 2 and 2 * a case

  def __rmul__(self, other):
    return self*other

  # exp function
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data*out.grad 
    out._backward = _backward
    
    return out

  ##power function for a constant
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "support"
    out = Value(self.data**other, (self,), f'**{other}')
    
    def _backward():
      self.grad += other*(self.data**(other-1))*out.grad 
    out._backward = _backward

    return out    

  # we need to define a power function as well 
  def __truediv__(self, other):
    return self*other**-1

  # for activation function
  def tanh(self):
    n = self.data
    t = (math.exp(2*n)-1)/(math.exp(2*n)+1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad = (1-t**2) * out.grad
    out._backward = _backward

    return out
  
  def backward(self): #topologically sorted back prop
    topo = [] # Empty array containing the nodes in right order
    visited = set() 
    def build_topo(v):
      if v not in visited:
        visited.add(v)
      for child in v._prev:
        build_topo(child)
      topo.append(v)

    build_topo(self)

    self.grad = 1.0
    ## Then implement _backward() method on this array sorted in topological order
    for node in reversed(topo):
      node._backward()

In [22]:
b - a
# Remains a bug

NameError: ignored

In [24]:
# using pytorch for the same ops
import torch
x1 = torch.Tensor([2.0]).double() ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double() ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double() ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double() ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double() ; b.requires_grad = True

n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

0.7071066904050358


In [25]:
# Implementing NNs basic
import random
class Neuron:

  # nin is number of inputs to the neurons
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x): # Similar to a constructor to call the calls and underlying methods

    ## zip transforms the inputs to axb dimension array with each row as a tuple of wi and xi for efficient operations
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out


x = [2.0, 3.0]
n = Neuron(2)
n(x)

Value(data=-0.6011338133373041)

In [26]:
# Implementing NNs as MLP as 1 Layer
import random
class Neuron:

  # nin is number of inputs to the neurons
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x): # Similar to a constructor to call the calls and 
                         # underlying methods

    ## zip transforms the inputs to axb dimension array with each row as a tuple 
    ## of wi and xi for efficient operations
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

# layer made up of individual neurons, each having nin inputs and nout outputs
# A layer has neurons as total nuerons, hence to build a complete connected  
#   layer, we iterate over each neurons, return the output of that neuron and  
#   store in "outs" array, and return outs to the next layer as input

class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs


x = [2.0, 3.0]
# n = Neuron(2)
n = Layer(2, 3)
n(x)

# Outputs are 3 outputs, from the 3 neurons in the final layer

[Value(data=-0.10455989764133997),
 Value(data=-0.5223653684280234),
 Value(data=0.9999160723656918)]

In [45]:
# Implementing NNs as MLP as n Layer with nlay neurons list per layer
import random
class Neuron:

  # nin is number of inputs to the neurons
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x): # Similar to a constructor to call the calls and 
                         # underlying methods

    ## zip transforms the inputs to axb dimension array with each row as a tuple 
    ## of wi and xi for efficient operations
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

# layer made up of individual neurons, each having nin inputs and nout outputs
# A layer has neurons as total nuerons, hence to build a complete connected  
#   layer, we iterate over each neurons, return the output of that neuron and  
#   store in "outs" array, and return outs to the next layer as input

class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

class MLP:
  def __init__(self, nin, nouts): # Taking nin as number of inputs to MLP and nouts as a list of number of neurons per layer(including final layer) for MLP
    size_MPL = [nin] + nouts #Converting nin and nouts to the list and adding as first layer also included in MLP
    self.layers = [Layer(size_MPL[i], size_MPL[i+1]) for i in range(len(nouts))] # Feeding output of one layer to the next consecutive layer

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

x = [2.0, 3.0, -1]
n = MLP(3, [4, 4, 1])

n(x)

# Outputs are 1 output, from the 1 neuron in the final layer

Value(data=-0.7270090561600445)

In [ ]:
draw_dot(n(x))

In [59]:
# Let's collect all parameters (w and biases) of every for, back prop to operate better
# Implementing NNs as MLP as n Layer with nlay neurons list per layer

import random
class Neuron:

  # nin is number of inputs to the neurons
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x): # Similar to a constructor to call the calls and 
                         # underlying methods

    ## zip transforms the inputs to axb dimension array with each row as a tuple 
    ## of wi and xi for efficient operations
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

# layer made up of individual neurons, each having nin inputs and nout outputs
# A layer has neurons as total nuerons, hence to build a complete connected  
#   layer, we iterate over each neurons, return the output of that neuron and  
#   store in "outs" array, and return outs to the next layer as input

class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def params(self):
    return [p for neurons in self.neurons for p in neurons.parameters()]

class MLP:
  def __init__(self, nin, nouts): # Taking nin as number of inputs to MLP and nouts as a list of number of neurons per layer(including final layer) for MLP
    size_MPL = [nin] + nouts #Converting nin and nouts to the list and adding as first layer also included in MLP
    self.layers = [Layer(size_MPL[i], size_MPL[i+1]) for i in range(len(nouts))] # Feeding output of one layer to the next consecutive layer

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def params_MLP(self):
    return [p for layers in self.layers for p in layers.params()]

x = [2.0, 3.0, -1]
n = MLP(3, [4, 4, 1])
n(x)


# Outputs are 1 output, from the 1 neuron in the final layer

Value(data=0.09219236495994562)

In [61]:
# Taking a dummy dataset
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0]
ypred = [n(x) for x in xs]

loss = sum((yout-ygt)**2 for ygt, yout in zip(ys, ypred))
loss.backward()
loss

Value(data=6.742590684127812)

In [ ]:
# Implementing Gradient descent on every parameter in the list of parameters of MLP
alpha = 0.01
for p in n.params_MLP():
  p.data += alpha*p.grad